In [1]:
# Use autoplot 207's code for this app
import sys
sys.path.insert(0, "/opt/iem/htdocs/plotting/auto/scripts200/")
from p207 import load_data, compute_grid_bounds, add_zeros, do_analysis, USEME
from pyiem.plot import MapPlot, nwssnow
from pyiem.util import get_dbconn, utc
import numpy as np

In [9]:
STORM_NUMBER = 3
TITLE = "25-26 October 2020"
SUBTITLE = "8 AM 26 October 2020"
sts = utc(2020, 10, 26, 0)
ets = utc(2020, 10, 26, 23, 0)
# Get available data
ctx = {'coop': 'yes', 'csector': 'IA', 't': 'state', 'sz': 25, 'z': 'yes', 'f': 'linear'}
df = load_data(ctx, sts, ets)
# figure out our grid bounds
ctx["bnds2163"] = compute_grid_bounds(ctx)
# add zeros and QC
df = add_zeros(df, ctx)

In [10]:
def workflow(ctx, df, isfinal=False, lower=0, upper=2):
    # do gridding
    df2 = df[df[USEME]]
    lons, lats, vals = do_analysis(df2, ctx)
    mp = MapPlot(
        sector="state",
        state=ctx["csector"],
        axisbg="white",
        title="%s - IEM Snowfall Total Analysis" % (TITLE, ),
        subtitle=("Snowfall totals till %s, "
                  "IEM 2020-2021 Winter Storm #%s") % (SUBTITLE, STORM_NUMBER),
    )
    mp.contourf(
        lons, lats, vals, np.array([0.1, 1, 2, 3, 4, 6, 8, 12, 18, 24, 30, 36]), cmap=nwssnow(), clip_on=True)
    df_useme_plot = df2[(df2['val']>=lower)&(df2['val'] < upper)]
    print(df_useme_plot)
    mp.drawcounties()
    if isfinal:
        mp.drawcities()
    else:
        mp.plot_values(df_useme_plot['lon'], df_useme_plot['lat'], df_useme_plot['val'],
            '%s', labels=df_useme_plot['nwsli'].values,
            textsize=10, labeltextsize=10, labelbuffer=0)
    return mp

In [15]:
cull = ['Z5', 'CINI4', 'OSEI4']
if cull:
    df.loc[df['nwsli'].isin(cull), USEME] = False
# df.loc[df[df["state"] == "MN"].index, USEME] = False
hardcode = [
    ['DUBI4', 0.1],
]
for nwsli, val in hardcode:
    df.loc[df['nwsli'] == nwsli, 'val'] = val

ctx['csector'] = "IA"
mp = workflow(ctx, df, isfinal=True, lower=0, upper=0.1)
res = mp.postprocess(filename='201026.png')
mp.close()

     state  wfo     val        lon        lat                             geo  \
520     IA  DMX  0.0001 -93.950800  40.623300  POINT (510344.175 -467845.090)   
522     IA  ARX  0.0001 -91.523000  42.811500  POINT (690068.855 -207518.151)   
526     IA  ARX  0.0001 -91.530000  42.680000  POINT (690983.458 -222140.482)   
528     IA  ARX  0.0001 -91.875100  42.680100  POINT (662946.976 -224987.271)   
530     IA  ARX  0.0001 -91.453700  42.775400  POINT (696095.426 -210928.088)   
531     IL  DVN  0.0001 -90.548000  41.399900  POINT (786613.638 -354986.323)   
533     IL  DVN  0.0001 -89.900000  41.250000  POINT (842189.464 -365143.856)   
534     IL  DVN  0.0001 -89.616700  42.300000  POINT (851350.765 -246146.121)   
535     IL  DVN  0.0001 -90.250000  42.333300  POINT (799311.450 -248851.961)   
551     MO  EAX  0.0001 -94.834100  40.345800  POINT (437804.033 -503699.378)   
553     MO  EAX  0.0001 -94.497700  40.464000  POINT (465425.071 -488746.829)   
558     MO  EAX  0.0001 -94.